In [ ]:
import readline

def input_with_prefill(prompt, text):
    def hook():
        readline.insert_text(text)
        readline.redisplay()
    readline.set_pre_input_hook(hook)
    result = input(prompt)
    readline.set_pre_input_hook()
    return result


print(input_with_prefill('Enter something: ', 'default text'))

In [ ]:
import shutil
if shutil.which('pilercr'):
    print('python3 is installed')
else:
    print('python3 is not installed')

In [ ]:
import os
tool="minced"
input_dir = '/shares/CIBIO-Storage/CM/scratch/users/isacco.cenacchi/Tirocinio/samples/MAGs_mini'
output_dir = "/shares/CIBIO-Storage/CM/scratch/users/isacco.cenacchi/Tirocinio/out/MAGs_mini_CRISPRDetect3_20241001161255"
# output_file = os.path.join(input_dir.removesuffix(os.path.basename(input_dir)), os.path.basename(args.out))
os.path.basename(input_dir)+'_'+tool+'_parsed.tsv'
os.path.basename(output_dir)+'_parsed.tsv'
os.path.join(output_dir, os.path.basename(input_dir)+'_'+tool+'_parsed.tsv')
# input_dir.removesuffix(os.path.basename(input_dir))

In [ ]:
repeats = ["..A..................................CC.", ".....................................---", "GG............-......................--."]
reference = "AAGTTTCCGTCCCCTTTCGGGGAATCATTTAGAAAAT--A"
line="   145662      33    93.9      33  TAGTATTTAA    ...............T................    AGAAAACCGCATAAGGACCGATACCACATTACA"
stringa = ".--.-...............T................"
count = stringa[:stringa.find(".")].count("-")
seqs = line.split()
start=int(seqs[0]) - seqs[5][:seqs[5].find(".")].count("-")
def develop_repeats(repeats, reference):
    developed_repeats = []
    for repeat in repeats:
        repeat = list(repeat)
        for i in range(len(reference)):
            if repeat[i] == '.': 
                repeat[i] = reference[i]
        repeat = ''.join(repeat)
        repeat = repeat.replace('-', '')
        developed_repeats.append(repeat)
    return developed_repeats

develop_repeats(repeats, reference)
start
# print(develop_repeats(repeats, reference))

In [ ]:
class CRISPR:
    '''
    A class used to represent a CRISPR array

    Attributes:
        file_name (str): the name of the file that the CRISPR was found in (MAG name)
        contig_name (str): the name of the contig that the CRISPR was found in
        start (int): Position of the first base in the CRISPR (one-indexed, inclusive)
        end (int): position of the last base in the CRISPR (one-indexed, inclusive)
        spacers (list): a list of the ordered spacers in the CRISPR
        repeats (list): a list of the ordered repeats in the CRISPR
        flankers (dict): a dictionary with the left and right flankers of the CRISPR
    
    Methods:
        __init__(): Constructor
        __len__(): Returns the length of the CRISPR calculated as the sum of the lengths of the spacers and repeats
        __bool__(): Returns True if the CRISPR is valid, False otherwise
        setFile_name(file_name): Sets the file_name attribute
        setContig_name(contig_name): Sets the contig_name attribute
        setStart(start): Sets the start attribute
        setEnd(end): Sets the end attribute
        addRepeat(repeat): Adds a repeat to the repeats list
        addSpacer(spacer): Adds a spacer to the spacers list
        setFlankerLeft(left): Sets the left flanker
        setFlankerRight(right): Sets the right flanker
    '''
    def __init__(self, file_name=None, contig_name=None, start=None, end=None):
        self.file_name = file_name
        self.contig_name = contig_name
        self.start = start
        self.end = end
        self.spacers = []
        self.repeats = []
        self.flankers = {'left': '', 'right': ''}
    
    def __repr__(self):
        return f'<CRISPR object: (\n{self.file_name}\n{self.contig_name}\n{self.start}\n{self.end}\n{self.spacers}\n{self.repeats}\n{self.flankers})>\n'
    
    def __str__(self):
        return f'f_name: {self.file_name}\ncontig: {self.contig_name}\nstart: {self.start}\nend: {self.end}\nspacers: {self.spacers}\nrepeats: {self.repeats}\nflankers: {self.flankers}\n'
    
    def __len__(self):
        return sum(len(spacer) for spacer in self.spacers) + sum(len(repeat) for repeat in self.repeats)
    
    def __bool__(self):
        return (isinstance(self.file_name, str) and self.file_name != '' and
                isinstance(self.contig_name, str) and self.contig_name != '' and
                isinstance(self.start, int) and self.start >= 0 and
                isinstance(self.end, int) and self.end >= self.start and
                len(self) == (self.end - self.start + 1))
    
    def __eq__(self, other):
        return self.file_name == other.file_name and self.contig_name == other.contig_name and self.start == other.start and self.end == other.end
    
    def setFile_name(self, file_name):
        self.file_name = file_name
    
    def setContig_name(self, contig_name):
        self.contig_name = contig_name
    
    def setStart(self, start):
        self.start = start

    def setEnd(self, end):
        self.end = end
    
    def addRepeat(self, repeat):
        self.repeats.append(repeat)
    
    def addSpacer(self, spacer):
        self.spacers.append(spacer)

    def setFlankerLeft(self, left):
        self.flankers['left'] = left

    def setFlankerRight(self, right):
        self.flankers['right'] = right

def develop_repeats(repeats, reference):
    developed_repeats = []
    for repeat in repeats:
        repeat = list(repeat)
        for i in range(len(reference)):
            if repeat[i] == '.': 
                repeat[i] = reference[i]
        repeat = ''.join(repeat)
        repeat = repeat.replace('-', '')
        developed_repeats.append(repeat)
    return developed_repeats

def parse_minced(file_path):
    crisprs = []
    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith("Sequence '"):
                contig_name = line.split("'")[1]
            elif line.startswith("CRISPR"):
                start, end = map(int, line.split()[3:6:2]) # Take from 4th to 6th element, step 2
                crispr_tmp = CRISPR(file_name=file_path.split('/')[-1].split('.')[0], contig_name=contig_name, start=start, end=end)
            elif line[:1].isdigit():
                seqs = line.split()
                if len(seqs) == 7:
                    crispr_tmp.addRepeat(seqs[1])
                    crispr_tmp.addSpacer(seqs[2])
                if len(seqs) == 2:
                    crispr_tmp.addRepeat(seqs[1])
            # Save the instance
            elif line.startswith("Repeats"):
                if bool(crispr_tmp):
                    crisprs.append(crispr_tmp)
                else:
                    raise ValueError(f"Invalid CRISPR format in file {file_path}")
    return crisprs

def parse_CRISPRDetect3(file_path):
    crisprs = []
    with open(file_path, 'r') as file:
        crispr_tmp = None
        for line in file:
            line = line.strip()
            if line.startswith("Array"):
                if crispr_tmp is not None:
                    raise ValueError(f"CRISPR not finished in file {file_path}, contig {crispr_tmp.contig_name}")
            elif line.startswith(">"):
                contig_name = line[1:].split()[0]
            elif line[:1].isdigit():
                seqs = line.split()
                if len(seqs) == 7 and crispr_tmp is None: # first line: flankerLeft - repeat - spacer
                    start=int(seqs[0]) + seqs[5][:seqs[5].find(".")].count("-") # adjust start position if there are gaps in the repeat
                    crispr_tmp = CRISPR(file_name=file_path.split('/')[-1].split('.')[0], contig_name=contig_name, start=start, end=None)
                    crispr_tmp.setFlankerLeft(seqs[4])
                    repeats = [seqs[5]]
                    crispr_tmp.addSpacer(seqs[6])
                elif len(seqs) == 7 and crispr_tmp is not None: # next line: repeat - spacer
                    repeats.append(seqs[5])
                    crispr_tmp.addSpacer(seqs[6])
                elif len(seqs) == 6 and crispr_tmp is not None: # last line: repeat - flankerRight
                    repeats.append(seqs[4])
                    crispr_tmp.setFlankerRight(seqs[5])
                elif len(seqs) == 4 and crispr_tmp is not None: # consensus repeat
                    consensus = seqs[3]
                    for repeat in develop_repeats(repeats, consensus):
                        crispr_tmp.addRepeat(repeat)
                    repeats = []
                    crispr_tmp.setEnd(crispr_tmp.start + len(crispr_tmp) - 1)
                    if bool(crispr_tmp):
                        crisprs.append(crispr_tmp)
                        crispr_tmp = None
                    else:
                        raise ValueError(f"Invalid CRISPR format in file {file_path}")
            elif line.startswith("SUMMARY"):
                break
        return crisprs


In [13]:
line='>NODE_6421_length_4372_cov_9.0278		Array_Orientation: Unconfirmed'
line = line.strip()
contig_name = line[1:].split()[0]
contig_name
line[:1]

'>'